# PyTorch로 시작하는 딥러닝 입문

## 4장 : SoftMax Regression

https://wikidocs.net/59425

## SoftMax Regression

Logistic Regression이 2개의 선택지 중 1개를 고르는 Binary Classification이라면, SoftMax Regression은 3개 이상의 선택지 중 1개를 고르는 Multi-Class Classification입니다.

### Logistic Regression
로지스틱 회귀에서 시그모이드 함수는 예측값을 0과 1 사이의 값으로 만듭니다.
예를 들어 스팸 메일 분류기를 로지스틱 회귀로 구현하였을 때, 출력이 0.75라면 스팸일 확률은 75%, 스팸이 아닐 확률이 25%가 됩니다.

![](https://wikidocs.net/images/page/59427/%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%ED%9A%8C%EA%B7%80.PNG)
H(x) = sigmoid(Wx+b)

### SoftMax Regression
소프트맥스 회귀는 확률의 합이 1이 되는 위의 아이디어를 다중 클래스 분류 문제에 적용합니다. 각 클래스, 각 선택지마다 확률을 할당하며 이 때 확률의 합은 1이 되어야 합니다. 각 선택지가 정답일 확률로 표현됩니다.
![](https://wikidocs.net/images/page/59427/%EC%86%8C%ED%94%84%ED%8A%B8%EB%A7%A5%EC%8A%A4%ED%9A%8C%EA%B7%80.PNG)

결국 소프트맥스 회귀는 선택지의 개수만큼의 차원을 가지는 벡터를 만들고, 해당 벡터가 벡터의 모든 원소의 합이 1이 되도록 원소들의 값을 변환시키는 함수를 지나게 만들어야 합니다.

## SoftMax Function

SoftMax Function은 분류해야하는 클래스의 총 개수를 k라고 할 때, k 차원의 벡터를 입력받아 각 클래스에 대한 확률을 추정합니다.

### 1) Softmax Function Understanding
k 차원의 벡터에서 i 번째 원소를 z, i 번째 클래스가 정답일 확률을 p로 나타낸다고 하였을 때 소프트맥스 함수는 다음과 같이 정의합니다.
$
p_{i}=\frac{e^{z_{i}}}{\sum_{j=1}^{k} e^{z_{j}}}\ \ for\ i=1, 2, ... k
$

예시에서 주어진 문제의 경우 k=3이므로 3차원 벡터 z=[z1, z2, z3]의 입력을 받으면 소프트맥스 함수는 아래와 같은 출력을 반환합니다.
$
softmax(z)=[\frac{e^{z_{1}}}{\sum_{j=1}^{3} e^{z_{j}}}\ \frac{e^{z_{2}}}{\sum_{j=1}^{3} e^{z_{j}}}\ \frac{e^{z_{3}}}{\sum_{j=1}^{3} e^{z_{j}}}] = [p_{1}, p_{2}, p_{3}] = \hat{y} = \text{예측값}
$

p1, p2, p3는 각각 클래스가 정답일 확률을 의미하며 p1+p2+p3 = 1이 됩니다.
분류하고자 하는 클래스가 k개인 경우 k차원의 벡터를 입력 받아서 모든 벡터 원소 값을 0과 1 사이의 값으로 변경하여 다시 k차원의 벡터를 리턴하는 과정입니다.

### 2) Picture For Further Understanding
![](https://wikidocs.net/images/page/35476/softmax1_final_final.PNG)

소프트맥스 함수는 4개의 독립변수를 가지므로 이는 모델이 4차원 벡터를 입력으로 받음을 의미합니다. 그러나 최종적으로 소프트맥스 함수의 입력으로 사용되는 벡터는 분류하고자 하는 클래스의 개수가 되어야 하므로 어떠한 가중치 연산을 통해 3차원 벡터로 변환되어야 합니다.

![](https://wikidocs.net/images/page/35476/softmaxbetween1and2.PNG)

이제부터 SoftMax Regression Model을 End-to-End 구현하겠습니다!

## SoftMax Regression Model(Low Level)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

C:\Users\82104\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
torch.manual_seed(1)

In [3]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [4]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


x_train의 크기는 8x4이며, y_train의 크기는 8x1이므로 y_train에 one-hot encoding한 결과는 8x3의 크기를 가져야 합니다.

인코딩 된 y_train의 크기는 8x3이므로, w 행렬의 크기는 4x3이어야 합니다.

w와 b를 선언하고, optimizer로는 경사하강법을 사용하며, learning rate는 0.1로 설정합니다.

In [5]:
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [6]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [7]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## SoftMax Regression Model(High Level)

In [8]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## SoftMax Regression with nn.Module

In [9]:
# 모델을 선언 및 초기화. 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3.
model = nn.Linear(4, 3)

In [10]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.658891
Epoch  200/1000 Cost: 0.573444
Epoch  300/1000 Cost: 0.518151
Epoch  400/1000 Cost: 0.473266
Epoch  500/1000 Cost: 0.433516
Epoch  600/1000 Cost: 0.396563
Epoch  700/1000 Cost: 0.360914
Epoch  800/1000 Cost: 0.325392
Epoch  900/1000 Cost: 0.289178
Epoch 1000/1000 Cost: 0.254148


## SoftMax Regression with Class

In [11]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)

In [12]:
model = SoftmaxClassifierModel()

In [13]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.647903
Epoch  200/1000 Cost: 0.564643
Epoch  300/1000 Cost: 0.511043
Epoch  400/1000 Cost: 0.467249
Epoch  500/1000 Cost: 0.428281
Epoch  600/1000 Cost: 0.391924
Epoch  700/1000 Cost: 0.356742
Epoch  800/1000 Cost: 0.321577
Epoch  900/1000 Cost: 0.285617
Epoch 1000/1000 Cost: 0.250818


In [14]:
print(list(model.parameters()))

[Parameter containing:
tensor([[-3.1238, -0.3840,  2.2948, -0.5021],
        [ 0.5727, -0.2390, -0.5843,  1.2551],
        [ 2.4460,  0.3490, -1.9904, -0.1017]], requires_grad=True), Parameter containing:
tensor([-2.1903, -1.1847,  3.4679], requires_grad=True)]
